In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, mean_squared_error, root_mean_squared_error, classification_report, f1_score, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
import pandas as pd
import numpy as np
import re
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier, plot_tree, plot_importance
import time
import random
from shaphypetune import BoostSearch, BoostRFE, BoostRFA, BoostBoruta
from hyperopt import hp, Trials
from scipy import stats

/lustre/scratch125/casm/team215mg/pg21_rotation/software/miniforge3/envs/xgboost/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coefficients = pd.DataFrame()
timepoints = ['24', '48', '72']

for rep in range(1,11):
    for time in timepoints:
        temp_coeff = pd.read_csv(f'/lustre/scratch125/casm/team215mg/pg21_rotation/Coefficients_CellDrift/original/selection_rep_{rep}/Contrast_Coefficients_time_{time}.txt', sep = '\t')
        temp_coeff['contrast'] = temp_coeff['contrast'] + f'_{rep}'
        temp_coeff['time'] = time
        coefficients = pd.concat([coefficients, temp_coeff], ignore_index = True)

coefficients['cell_type'] = coefficients['cell_type'].str.split(',', n = 1).str.get(-1).str.split("'").str.get(-2)
coefficients['perturbation'] = coefficients['perturbation'].str.split(',', n = 1).str.get(0).str.split("'").str.get(-2)
coefficients['cell_type'] = coefficients['cell_type'].map({'Fetal':0, 'Stem':1, 'Cycling':2, 'Secretory':3, 'Stress':4, 'Respiration':5})

#coefficients = coefficients[coefficients['p_fdr'] < 0.05]
coefficients = coefficients[(coefficients['perturbation'] == 'MRTX1133_SHP099') | (coefficients['perturbation'] == 'MRTX1133_afatinib')]

coefficients

,contrast,cell_type,perturbation,mean,lci,uci,SE,z,p,p_fdr,pts_contrast,pts_reference,gene,time
1,Cycling_MRTX1133_SHP099-Cycling_DMSO_1,2,MRTX1133_SHP099,1.0593,1.7051,0.4136,0.3295,3.2152,0.0013,0.007800,0.219178,0.123333,ABAT,24
2,Cycling_MRTX1133_afatinib-Cycling_DMSO_1,2,MRTX1133_afatinib,1.0974,1.8266,0.3682,0.3721,2.9495,0.0032,0.016258,0.200000,0.123333,ABAT,24
6,Fetal_MRTX1133_SHP099-Fetal_DMSO_1,0,MRTX1133_SHP099,-0.6763,-0.2199,-1.1328,0.2329,-2.9040,0.0037,0.017794,0.350000,0.360000,ABAT,24
7,Fetal_MRTX1133_afatinib-Fetal_DMSO_1,0,MRTX1133_afatinib,-0.1212,0.2658,-0.5082,0.1975,-0.6139,0.5393,0.670136,0.430000,0.360000,ABAT,24
11,Stem_MRTX1133_SHP099-Stem_DMSO_1,1,MRTX1133_SHP099,0.3056,0.8021,-0.1909,0.2533,1.2065,0.2276,0.377337,0.240000,0.213333,ABAT,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733487,Respiration_MRTX1133_afatinib-Respiration_DMSO_10,5,MRTX1133_afatinib,0.0267,229754.7195,-229754.6662,117223.9361,0.0000,1.0000,1.000000,0.000000,0.000000,ZNF831,72
733491,Secretory_MRTX1133_SHP099-Secretory_DMSO_10,3,MRTX1133_SHP099,0.0239,277440.0052,-277439.9574,141553.6120,0.0000,1.0000,1.000000,0.000000,0.000000,ZNF831,72
733492,Secretory_MRTX1133_afatinib-Secretory_DMSO_10,3,MRTX1133_afatinib,-0.0707,681302.9532,-681303.0946,347609.9710,-0.0000,1.0000,1.000000,0.000000,0.000000,ZNF831,72
733496,Stress_MRTX1133_SHP099-Stress_DMSO_10,4,MRTX1133_SHP099,24.8449,107218.1735,-107168.4837,54691.4787,0.0005,0.9996,1.000000,0.010000,0.000000,ZNF831,72


In [3]:
pivot = pd.read_csv('/lustre/scratch125/casm/team215mg/pg21_rotation/all_pivot.tsv', sep = '\t')

In [4]:
pivot_24 = pivot[pivot['time'] == 24]
pivot_48 = pivot[pivot['time'] == 48]
pivot_72 = pivot[pivot['time'] == 72]
y_24 = pivot_24['cell_type'].to_numpy()
y_48 = pivot_48['cell_type'].to_numpy()
y_72 = pivot_72['cell_type'].to_numpy()

In [5]:
for gene in np.unique(coefficients['gene']):
    pivot_24[f'{gene}_delta'] = pivot_48[f'{gene}_delta'].values
    pivot_48[f'{gene}_delta'] = pivot_72[f'{gene}_delta'].values

/tmp/ipykernel_3988171/1922417773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_24[f'{gene}_delta'] = pivot_48[f'{gene}_delta'].values
/tmp/ipykernel_3988171/1922417773.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_48[f'{gene}_delta'] = pivot_72[f'{gene}_delta'].values


In [6]:
pivot_24.drop(['Unnamed: 0', 'contrast', 'cell_type', 'time'], axis = 1, inplace = True)
X_24 = pivot_24.to_numpy()
pivot_48.drop(['Unnamed: 0', 'contrast', 'cell_type', 'time'], axis = 1, inplace = True)
X_48 = pivot_48.to_numpy()
pivot_72.drop(['Unnamed: 0', 'contrast', 'cell_type', 'time'], axis = 1, inplace = True)
X_72 = pivot_72.to_numpy()
X_48

/tmp/ipykernel_3988171/768850682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_24.drop(['Unnamed: 0', 'contrast', 'cell_type', 'time'], axis = 1, inplace = True)
/tmp/ipykernel_3988171/768850682.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_48.drop(['Unnamed: 0', 'contrast', 'cell_type', 'time'], axis = 1, inplace = True)
/tmp/ipykernel_3988171/768850682.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_

array([[ 2.2450e+00,  2.2790e+00,  2.7905e+00, ..., -1.6599e+00,
        -3.5490e-01,  0.0000e+00],
       [ 3.2930e+00,  3.2890e+00,  7.1580e-01, ..., -3.4737e+00,
         9.9380e-01,  0.0000e+00],
       [ 3.7158e+00,  2.4421e+00,  1.9676e+00, ..., -1.4243e+00,
         3.9620e-01,  1.0000e-04],
       ...,
       [ 2.3563e+00,  1.5860e+00,  8.6050e-01, ..., -1.8477e+00,
        -1.5189e+00,  0.0000e+00],
       [ 1.5594e+00,  4.5710e-01,  1.3748e+00, ..., -7.2690e-01,
        -2.5763e+00,  0.0000e+00],
       [ 1.7211e+00,  2.5660e-01,  1.0695e+00, ...,  2.9550e-01,
        -2.2709e+00,  1.0000e-04]])

In [7]:
X_24_train, X_24_test, y_24_train, y_24_test = train_test_split(pivot_24, y_24, test_size = 0.2)
X_48_train, X_48_test, y_48_train, y_48_test = train_test_split(pivot_48, y_48, test_size = 0.2)
X_72_train, X_72_test, y_72_train, y_72_test = train_test_split(pivot_72, y_72, test_size = 0.2)

In [11]:
xgb_tuned = XGBClassifier(learning_rate = 0.2,
                          max_depth = 18,
                          n_estimators = 421,
                          subsample = 0.2,
                          reg_alpha = 0,
                          reg_lambda = 0,
                          early_stopping_rounds = 20,
                          device = 'cuda')
xgb_tuned.fit(pivot_24, y_24, eval_set = [(pivot_72, y_72)])

[0]	validation_0-mlogloss:1.55960
[1]	validation_0-mlogloss:1.42715
[2]	validation_0-mlogloss:1.35743


/lustre/scratch125/casm/team215mg/pg21_rotation/software/miniforge3/envs/xgboost/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [10:04:28] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1737698414298/work/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


[3]	validation_0-mlogloss:1.30573
[4]	validation_0-mlogloss:1.26003
[5]	validation_0-mlogloss:1.20005
[6]	validation_0-mlogloss:1.17664
[7]	validation_0-mlogloss:1.12235
[8]	validation_0-mlogloss:1.12057
[9]	validation_0-mlogloss:1.13223
[10]	validation_0-mlogloss:1.10507
[11]	validation_0-mlogloss:1.09333
[12]	validation_0-mlogloss:1.09681
[13]	validation_0-mlogloss:1.10362
[14]	validation_0-mlogloss:1.13517
[15]	validation_0-mlogloss:1.15907
[16]	validation_0-mlogloss:1.16159
[17]	validation_0-mlogloss:1.16184
[18]	validation_0-mlogloss:1.19406
[19]	validation_0-mlogloss:1.19579
[20]	validation_0-mlogloss:1.19603
[21]	validation_0-mlogloss:1.17994
[22]	validation_0-mlogloss:1.17631
[23]	validation_0-mlogloss:1.16964
[24]	validation_0-mlogloss:1.17402
[25]	validation_0-mlogloss:1.16468
[26]	validation_0-mlogloss:1.16196
[27]	validation_0-mlogloss:1.15178
[28]	validation_0-mlogloss:1.15743
[29]	validation_0-mlogloss:1.14728
[30]	validation_0-mlogloss:1.15824
[31]	validation_0-mlogloss:

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=20,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=18, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=421, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [9]:
param_dist_hyperopt = {
    'max_depth': 10 + hp.randint('num_leaves', 5),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample': hp.uniform('hyp_subsample', 0.1, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5)
}

In [11]:
model = BoostRFE(xgb_tuned, param_grid = param_dist_hyperopt, n_iter = 8,
                 min_features_to_select = 1, step = 1, n_jobs = 256,
                 sampling_seed = 0)
model.fit(pivot_24, y_24, eval_set = [(pivot_72, y_72)], trials = Trials(), verbose = 0)


8 trials detected for ('max_depth', 'learning_rate', 'subsample', 'reg_alpha', 'reg_lambda')



KeyboardInterrupt: 

In [13]:
build_info = xgb.build_info()
for name in sorted(build_info.keys()):
    print(f'{name}: {build_info[name]}')

BUILTIN_PREFETCH_PRESENT: True
DEBUG: False
GCC_VERSION: [13, 3, 0]
MM_PREFETCH_PRESENT: True
USE_CUDA: False
USE_DLOPEN_NCCL: False
USE_FEDERATED: False
USE_NCCL: False
USE_OPENMP: True
USE_RMM: False
libxgboost: /lustre/scratch125/casm/team215mg/pg21_rotation/software/miniforge3/envs/xgboost/lib/libxgboost.so


In [14]:
xgb_tuned

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=20,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=18, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=421, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)